# Generate captions for images

Automatically create descriptive captions for images using AI vision models.

## Problem

You have a collection of images that need captions—for accessibility, SEO, content management, or searchability. Writing captions manually doesn't scale.

| Use case | Images | Need |
|----------|--------|------|
| E-commerce catalog | 10K products | SEO-friendly descriptions |
| Photo library | 50K photos | Searchable metadata |
| Social media | 1K posts | Alt text for accessibility |

## Solution

**What's in this recipe:**
- Generate captions using OpenAI's vision models
- Customize caption style (short, detailed, SEO-focused)
- Process images in batch automatically

You add a computed column that sends each image to a vision model with a captioning prompt. New images are captioned automatically on insert.

### Setup

In [ ]:
%pip install -qU pixeltable openai

In [ ]:
import os
import getpass

if 'OPENAI_API_KEY' not in os.environ:
    os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key: ')

In [ ]:
import pixeltable as pxt
from pixeltable.functions.openai import vision

### Load images

In [ ]:
# Create a fresh directory
pxt.drop_dir('caption_demo', force=True)
pxt.create_dir('caption_demo')

In [ ]:
# Create table for images
images = pxt.create_table('caption_demo.images', {'image': pxt.Image})

In [ ]:
# Insert sample images
image_urls = [
    'https://raw.githubusercontent.com/pixeltable/pixeltable/main/docs/resources/images/000000000036.jpg',
    'https://raw.githubusercontent.com/pixeltable/pixeltable/main/docs/resources/images/000000000090.jpg',
    'https://raw.githubusercontent.com/pixeltable/pixeltable/main/docs/resources/images/000000000106.jpg',
]

images.insert([{'image': url} for url in image_urls])

In [ ]:
# View images
images.collect()

### Generate captions

Add a computed column that generates captions using the vision model:

In [ ]:
# Add caption using OpenAI vision
images.add_computed_column(
    caption=vision(
        images.image,
        prompt='Write a concise, descriptive caption for this image in one sentence.',
        model='gpt-4o-mini'
    )
)

In [ ]:
# View images with captions
images.select(images.image, images.caption).collect()

### Different caption styles

You can generate multiple caption styles for different uses:

In [ ]:
# Add alt text for accessibility (brief)
images.add_computed_column(
    alt_text=vision(
        images.image,
        prompt='Write a brief alt text for this image (under 125 characters) for screen readers.',
        model='gpt-4o-mini'
    )
)

In [ ]:
# Add detailed description
images.add_computed_column(
    description=vision(
        images.image,
        prompt='Describe this image in detail, including objects, colors, setting, and mood.',
        model='gpt-4o-mini'
    )
)

In [ ]:
# View all caption types
images.select(images.image, images.caption, images.alt_text).collect()

## Explanation

**Caption prompt patterns:**

| Use case | Prompt style |
|----------|--------------|
| General | "Write a concise caption" |
| Accessibility | "Write alt text under 125 characters" |
| SEO | "Describe for search engines with keywords" |
| Social media | "Write an engaging caption with hashtags" |
| E-commerce | "Describe this product for a catalog" |

**Model selection:**

- `gpt-4o-mini`: Fast and affordable, good for most captioning tasks
- `gpt-4o`: Higher quality for complex images or detailed descriptions

## See also

- [Analyze images in batch](https://docs.pixeltable.com/howto/cookbooks/images/vision-batch-analysis) - Run custom prompts on images
- [Extract structured data from images](https://docs.pixeltable.com/howto/cookbooks/images/vision-structured-output) - Get JSON from images
- [Find similar images](https://docs.pixeltable.com/howto/cookbooks/search/search-similar-images) - Visual similarity search